In [1]:
import tensorflow as tf, numpy as np

In [2]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(2, activation='tanh'),
    tf.keras.layers.Dense(2,  activation='linear'),
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='mse')

In [3]:
model.fit(
    np.random.uniform(size=(1000, 2)),
    np.random.uniform(size=(1000, 2))
)

32/32 [==============================] - 0s 893us/step - loss: 0.6680


In [4]:
class CCode:
    
    def __init__(self, code, header):
        self.code = code
        self.header = header

In [5]:
class CArrayConstant(CCode):
    def __init__(self, name, a, const=False, declared=True):
        if not isinstance(a, np.ndarray):
            assert isinstance(a, tuple)
            a = np.zeros(a)
        assert len(a.shape) == 2, 'We assume 2D arrays.'
        shape_name = name + '_shape'
        shape_declaration = 'const int {name}[{size}]'.format(
            name=shape_name,
            size=len(a.shape),
        )
        
        shape = '{name} = {{{data}}}'.format(
            name=shape_name,
            data=', '.join(['%d' % s for s in a.shape]),
        )
        
        data_name = name
        data_declaration = '{const}{dtypename} {name}[{size}]'.format(
            const='const ' if const else '',
            dtypename='int' if a.dtype == 'int' else 'double',
            size=a.size,
            name=data_name,
        )
        data = '{name} = {{{data}}}'.format(
            name=data_name,
            data=', '.join(['%.60g' % f for f in a.ravel()]),
        )
        self.code = shape + ';\n' + data + ';\n'
        self.actual_header = shape_declaration + ';\n' + data_declaration + ';\n'
        self.header = self.actual_header if declared else ''
        self.name = name

In [6]:
pairs = []

In [7]:
batch_size = 2

In [8]:
pairs.append(
    CArrayConstant(
        'input', 
        (batch_size, int(model.input.shape[1])),
        const=False,
    )
)

In [9]:
statements = []

In [10]:
class Layer(CCode):

    def __init__(self, i, activation):
        actstr = str(activation).lower()
        relu = 'relu' in actstr
        tanh = 'tanh' in actstr
        linear = (not relu) and (not tanh)
        self.code = '''
        mmult(
            preactivation_{i}, preactivation_{i}_shape[0], preactivation_{i}_shape[1],
            kernel_{i}, kernel_{i}_shape[0], kernel_{i}_shape[1],
            xA_{i}
            );
        bias_add(
            xA_{i}, xA_{i}_shape[0], xA_{i}_shape[1],
            bias_{i},
            preactivation_{iPlusOne}
        );
        activate(
            preactivation_{iPlusOne}, 
            preactivation_{iPlusOne}_shape[0],
            preactivation_{iPlusOne}_shape[1],
            {act_type}
        );
        '''.format(
            i=i, iPlusOne=i+1,
            act_type=(0 if relu else (1 if tanh else 2)),
        )
        self.header = ''
        self.name = 'layer_%d' % i
        self.i = i

In [11]:
for i, layer in enumerate(model.layers):
    kernel, bias = [x.numpy() for x in layer.weights]
    pairs.append(CArrayConstant('kernel_%d' % i, kernel))
    act_shape = (batch_size, kernel.shape[1])
    pairs.append(CArrayConstant('xA_%d'  % i, act_shape, const=False))
    last_layer = i == len(model.layers) - 1
    pairs.append(CArrayConstant('preactivation_%d' % (i+1,), act_shape, const=False, declared=not last_layer))
    statements.append(Layer(i, layer.activation)),

In [12]:
print(pairs[-1].actual_header)

const int preactivation_2_shape[2];
double preactivation_2[4];



In [13]:
print(statements[-1].code)


        mmult(
            preactivation_1, preactivation_1_shape[0], preactivation_1_shape[1],
            kernel_1, kernel_1_shape[0], kernel_1_shape[1],
            xA_1
            );
        bias_add(
            xA_1, xA_1_shape[0], xA_1_shape[1],
            bias_1,
            preactivation_2
        );
        activate(
            preactivation_2, 
            preactivation_2_shape[0],
            preactivation_2_shape[1],
            2
        );
        


In [14]:
main_function_code = CCode('''
void MLP(
    double *preactivation_0, const int batch_size, const int n_inputs,
    double *preactivation_{nlayers}, const int n_outputs
    ) {{
    const int preactivation_0_shape[2] = {{batch_size, n_inputs}};
    const int preactivation_{nlayers}_shape[2] = {{batch_size, n_outputs}};
    {statements}
}}
'''.format(
    nlayers=len(model.layers),
    statements='\n'.join([st.code for st in statements])
),
'''
void MLP(
    double *preactivation_0, const int batch_size, const int n_inputs,
    double *preactivation_{nlayers}, const int n_outputs
);
'''.format(
    nlayers=len(model.layers),
    statements='\n'.join([st.code for st in statements])
)
)
print(main_function_code.header)


void MLP(
    double *preactivation_0, const int batch_size, const int n_inputs,
    double *preactivation_2, const int n_outputs
);



In [15]:
print(main_function_code.code)


void MLP(
    double *preactivation_0, const int batch_size, const int n_inputs,
    double *preactivation_2, const int n_outputs
    ) {
    const int preactivation_0_shape[2] = {batch_size, n_inputs};
    const int preactivation_2_shape[2] = {batch_size, n_outputs};
    
        mmult(
            preactivation_0, preactivation_0_shape[0], preactivation_0_shape[1],
            kernel_0, kernel_0_shape[0], kernel_0_shape[1],
            xA_0
            );
        bias_add(
            xA_0, xA_0_shape[0], xA_0_shape[1],
            bias_0,
            preactivation_1
        );
        activate(
            preactivation_1, 
            preactivation_1_shape[0],
            preactivation_1_shape[1],
            1
        );
        

        mmult(
            preactivation_1, preactivation_1_shape[0], preactivation_1_shape[1],
            kernel_1, kernel_1_shape[0], kernel_1_shape[1],
            xA_1
            );
        bias_add(
            xA_1, xA_1_shape[0], xA_1_shape[1],

In [22]:
global_code = '\n'.join([
    c.code for c in pairs if c.header != ''
])
print(global_code)

input_shape = {2, 2};
input = {0, 0, 0, 0};

kernel_0_shape = {2, 2};
kernel_0 = {0.4177353680133819580078125, -1.1437270641326904296875, -0.963105142116546630859375, -0.332814276218414306640625};

xA_0_shape = {2, 2};
xA_0 = {0, 0, 0, 0};

preactivation_1_shape = {2, 2};
preactivation_1 = {0, 0, 0, 0};

kernel_1_shape = {2, 2};
kernel_1 = {-1.11199855804443359375, -0.616218388080596923828125, -0.35410785675048828125, 1.08596217632293701171875};

xA_1_shape = {2, 2};
xA_1 = {0, 0, 0, 0};



In [24]:
all_code = '''
#include "MLP.h"
#include "mm_utils.h"

{globals}

{func}
'''.format(globals=global_code, func=main_function_code.code)
# print(all_code)

In [25]:
all_headers = '''
{globals}

{func}
'''.format(globals='\n'.join([c.header for c in pairs]), func=main_function_code.header)
# print(all_headers)


const int input_shape[2];
double input[4];

const int kernel_0_shape[2];
double kernel_0[4];

const int xA_0_shape[2];
double xA_0[4];

const int preactivation_1_shape[2];
double preactivation_1[4];

const int kernel_1_shape[2];
double kernel_1[4];

const int xA_1_shape[2];
double xA_1[4];




void MLP(
    double *preactivation_0, const int batch_size, const int n_inputs,
    double *preactivation_2, const int n_outputs
);


